# 本地模型部署(chatGLM 6b)  

In [ ]:
"""
    源自清华大学 github 网站上的案例（https://github.com/THUDM/ChatGLM-6B）

    调试量化 int4级别

    先保证本地安装调试完CUDA，然后创建conda 环境 进行环境配置 pip install -r requirements.txt -i https://mirror.sjtu.edu.cn/pypi/web/simple

"""
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch")

from transformers import AutoTokenizer, AutoModel
model_path = "D:\CodeLibrary\ChatGLM\chatglm26b"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# model = AutoModel.from_pretrained(model_path, trust_remote_code=True, device='cuda')
# 按需修改，目前只支持 4/8 bit 量化
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).quantize(4).half().cuda()
model = model.eval()

response, history = model.chat(tokenizer, "你好", history=[])
print(response + '\n')

# response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
# print(response + '\n')

# 上面这种模型 定义 是不能够放入到 `RetrievalQA` 中的，因为这里的 `model` 是 `AutoModel` 类型，并不是可以 `Runnable` 的

**这样部署的模型直接拿去用是不对的，会出现下面错误：**

```python  
ValidationError: 2 validation errors for LLMChain


llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
```

# 正确的 `model` 定义如下：(这个还有点小问题，后面研究)          

In [ ]:
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModel
from typing import Optional, List

class HuggingfaceModel(LLM):
    def __init__(self, model_name: str):
        self.model = AutoModel.from_pretrained(model_name, trust_remote_code=True).eval()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
        output = self.model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
        return self.tokenizer.decode(output[0], skip_special_tokens=True)

    @property
    def _identifying_params(self) -> dict:
        return {"model_name": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "huggingface"

    def generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> List[str]:
        callback_manager = CallbackManagerForLLMRun.get_instance()
        with callback_manager.as_context():
            callback_manager.on_llm_start({self._llm_type: len(prompts)})
            results = [self._call(prompt, stop) for prompt in prompts]
            callback_manager.on_llm_end({self._llm_type: len(prompts)})
        return results


# 下面这个是没问题的  

In [1]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch")

class ChatGLM_LLM(LLM):
    # 基于本地 InternLM 自定义 LLM 类
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model_path :str):
        # model_path: InternLM 模型路径
        # 从本地初始化模型
        super().__init__()
        print("正在从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(torch.bfloat16).cuda()
        self.model = self.model.eval()
        print("完成本地模型的加载")

    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        # 重写调用函数
        response, history = self.model.chat(self.tokenizer, prompt , history=[])
        return response
        
    @property
    def _llm_type(self) -> str:
        return "ChatGLM3-6B"

# langchain 集成  

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA
import torch

# 加载文件夹中的所有txt类型的文件
loader = DirectoryLoader(r'D:/Notes/NLP review', glob='**/*.md')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 指定 Hugging Face 的预训练模型名称
model_name = r"D:\CodeLibrary\ChatGLM\embedtext2vec"  # 示例模型名称，您可以根据需要更改

# 创建 HuggingFaceEmbeddings 对象
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# 将 document 通过 HuggingFace 的 embeddings 对象计算 embedding 向量信息并临时存入 Chroma 向量数据库，用于后续匹配查询
docsearch = Chroma.from_documents(split_docs, embeddings)


model_name = "D:\CodeLibrary\ChatGLM\chatglm26b"
hf_model = ChatGLM_LLM(model_name)


正在从本地加载模型...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

完成本地模型的加载


# 参考代码 
```python
store = Chroma.load("chroma_store", embeddings)

template = """基于以下信息来回答用户问题。                          
已知信息： 
{context} 
问题：
{question}"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt":prompt}
qa = RetrievalQA.from_chain_type(llm=chatglm, retriever=store.as_retriever(), chain_type="stuff",
                                chain_type_kwargs=chain_type_kwargs, return_source_documents=True)
```

In [4]:
# 创建问答对象
# qa = RetrievalQA.from_chain_type(llm=hf_model, chain_type="stuff", vectorstore=docsearch, return_source_documents=True)

qa = RetrievalQA.from_chain_type(
    llm=hf_model,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True
)

# 进行问答 Chain          
"""
    类的 __call__ 方法已经被弃用， 用 invoke 方法来代替 __call__ 方法    
"""
# result = qa({"query": "电影《红高粱》简介？"})
# print(result)

result = qa.invoke({"query": "电影《红高粱》简介？"})
print(result)


{'query': '电影《红高粱》简介？', 'result': '电影《红高粱》是一部由张艺谋执导的彩色故事片，于 1987 年 10 月 23 日在北京颐和园陶然亭畔放映。该电影改编自莫言的同名小说，讲述了 20 世纪 50 年代至 70 年代发生在一个中国北方小村庄中的故事。\n\n故事的主人公是充满生命力和热情的女性角色九儿（巩俐饰演），她是一个坚韧不拔、充满生命力的女性，她在困境中努力生存，并在家庭和社会中扮演着重要的角色。\n\n电影讲述了九儿和她身边的几个重要人物之间的复杂关系和情感纠葛，其中包括了她的丈夫、情人、兄弟和邻居等。这些人物在不同的历史背景下经历了不同的命运，而九儿则以其强大的意志力和勇气，不断地面对困境并保护自己和家人。\n\n《红高粱》以其深刻的社会意义、生动的人物形象和出色的表演，赢得了广泛的关注和赞誉，成为了一部经典的中国电影。', 'source_documents': [Document(page_content='RAG Fusion\n\nRAG Fusion 参考，还讲了 RRF算法\n\n概念\n\nRAG 融合（RAG Fusion）是指在 RAG 模型中将检索和生成两个阶段进行有效整合的过程。在传统的 RAG 模型中，首先进行文档检索以获取相关文档，然后使用生成模型根据这些文档生成答案。但在某些情况下，这种简单的串行方法可能无法充分利用检索和生成之间的互补性。\n\nRAG 融合的目的是在保留检索和生成各自优势的同时，更加有效地利用两者之间的关系，从而提高整体的性能。这可以通过以下几种方式来实现：\n\n交互式融合：在检索和生成之间建立一种交互式的机制，让它们可以相互影响和调整。例如，在生成阶段可以考虑将生成的内容作为反馈信息反过来影响检索过程，以提高检索的准确性。\n\n信息传递：在检索到相关文档后，将一些关键信息传递给生成模型，以帮助生成更加相关和准确的答案。这些信息可以是关键词、上下文信息等。\n\n多阶段生成：将生成过程分为多个阶段，每个阶段都与检索结果有关，逐步细化生成的内容。例如，先生成一个粗略的答案，然后根据检索到的文档进一步完善和细化答案。\n\n端到端训练：将检索和生成过程作为一个整体进行端到端的训练，以更好地优化两者之间的关系，使得模型能够更好地学习到检索和生成之间的互相影响。\n\n通过 RA

In [6]:
print(result.keys())

dict_keys(['query', 'result', 'source_documents'])


In [10]:
print(result['query'])

电影《红高粱》简介？


In [13]:
print(result['result'])

电影《红高粱》是一部由张艺谋执导的彩色故事片，于 1987 年 10 月 23 日在北京颐和园陶然亭畔放映。该电影改编自莫言的同名小说，讲述了 20 世纪 50 年代至 70 年代发生在一个中国北方小村庄中的故事。

故事的主人公是充满生命力和热情的女性角色九儿（巩俐饰演），她是一个坚韧不拔、充满生命力的女性，她在困境中努力生存，并在家庭和社会中扮演着重要的角色。

电影讲述了九儿和她身边的几个重要人物之间的复杂关系和情感纠葛，其中包括了她的丈夫、情人、兄弟和邻居等。这些人物在不同的历史背景下经历了不同的命运，而九儿则以其强大的意志力和勇气，不断地面对困境并保护自己和家人。

《红高粱》以其深刻的社会意义、生动的人物形象和出色的表演，赢得了广泛的关注和赞誉，成为了一部经典的中国电影。


In [15]:
print(result["source_documents"])

[Document(page_content='RAG Fusion\n\nRAG Fusion 参考，还讲了 RRF算法\n\n概念\n\nRAG 融合（RAG Fusion）是指在 RAG 模型中将检索和生成两个阶段进行有效整合的过程。在传统的 RAG 模型中，首先进行文档检索以获取相关文档，然后使用生成模型根据这些文档生成答案。但在某些情况下，这种简单的串行方法可能无法充分利用检索和生成之间的互补性。\n\nRAG 融合的目的是在保留检索和生成各自优势的同时，更加有效地利用两者之间的关系，从而提高整体的性能。这可以通过以下几种方式来实现：\n\n交互式融合：在检索和生成之间建立一种交互式的机制，让它们可以相互影响和调整。例如，在生成阶段可以考虑将生成的内容作为反馈信息反过来影响检索过程，以提高检索的准确性。\n\n信息传递：在检索到相关文档后，将一些关键信息传递给生成模型，以帮助生成更加相关和准确的答案。这些信息可以是关键词、上下文信息等。\n\n多阶段生成：将生成过程分为多个阶段，每个阶段都与检索结果有关，逐步细化生成的内容。例如，先生成一个粗略的答案，然后根据检索到的文档进一步完善和细化答案。\n\n端到端训练：将检索和生成过程作为一个整体进行端到端的训练，以更好地优化两者之间的关系，使得模型能够更好地学习到检索和生成之间的互相影响。\n\n通过 RAG 融合，可以更好地整合检索和生成两个阶段，充分发挥它们各自的优势，从而提高整体系统的性能和效果。\n\n流程图部分\n\n之前的 多重查询之后， 可能会存在很多相似文档，给大模型之前肯定还是需要做些处理，故上面流程图中，文档与大模型之间的部分，就是 Fusion。这里涉及到一个 算法(Reciprocal ran fusion)，这是一个对文档进行打分的一个算法\n\nreciprocal rank fusion\n\n案例流程图\n\n计算公式\n$$RRFscore(d\\in D)=\\sum_{r\\in R}\\frac1{k+r(d)} ,$$\n\n来源（https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf）', metadata={'source': 'D:\\Notes\\NLP review\\RAG related\\RAG

In [17]:
import inspect

print(inspect.getsource(RetrievalQA.from_chain_type))

    @classmethod
    def from_chain_type(
        cls,
        llm: BaseLanguageModel,
        chain_type: str = "stuff",
        chain_type_kwargs: Optional[dict] = None,
        **kwargs: Any,
    ) -> BaseRetrievalQA:
        """Load chain from chain type."""
        _chain_type_kwargs = chain_type_kwargs or {}
        combine_documents_chain = load_qa_chain(
            llm, chain_type=chain_type, **_chain_type_kwargs
        )
        return cls(combine_documents_chain=combine_documents_chain, **kwargs)



In [19]:
a = qa.invoke({'query':'什么是RAG'})
print(a['result'])

RAG (Replacement Token Detection) 是一种预训练任务，旨在通过对输入句子中的某些token进行替换，并要求模型预测哪些token被替换了来进行模型训练。它不使用掩码，而是从一个建议分布中采样词来替换输入，这解决了掩码带来的预训练和 fine-tune 不一致的问题。然后模型再训练一个判别器，来预测每个词是原始词还是替换词。RTD任务的目的是学习区分输入的词，它不使用掩码，而是从一个建议分布中采样词来替换输入，这解决了掩码带来的预训练和 fine-tune 不一致的问题。然后模型再训练一个判别器，来预测每个词是原始词还是替换词。


In [2]:
from langchain.llms.base import LLM
import inspect

print(inspect.getsource(LLM))

class LLM(BaseLLM):
    """Simple interface for implementing a custom LLM.

    You should subclass this class and implement the following:

    - `_call` method: Run the LLM on the given prompt and input (used by `invoke`).
    - `_identifying_params` property: Return a dictionary of the identifying parameters
        This is critical for caching and tracing purposes. Identifying parameters
        is a dict that identifies the LLM.
        It should mostly include a `model_name`.

    Optional: Override the following methods to provide more optimizations:

    - `_acall`: Provide a native async version of the `_call` method.
        If not provided, will delegate to the synchronous version using
        `run_in_executor`. (Used by `ainvoke`).
    - `_stream`: Stream the LLM on the given prompt and input.
        `stream` will use `_stream` if provided, otherwise it
        use `_call` and output will arrive in one chunk.
    - `_astream`: Override to provide a native async version of